With reference to mkturkan's repo: [Generative Agents](https://github.com/mkturkcan/generative-agents)

## Libs

In [1]:
!nvidia-smi

Sun Jul  9 07:12:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.4 MB/s eta 0:00:00


In [3]:
from transformers import pipeline

In [4]:
import networkx as nx

## Init Vars

In [5]:
low_memory = False
prompt = "Write an email about an alpaca that likes flan" # Check if the model works
if low_memory:
  model = pipeline(model="declare-lab/flan-alpaca-xl", device=0)
else:
  model = pipeline(model="declare-lab/flan-alpaca-gpt4-xl", device=0)
model(prompt, max_length=128, do_sample=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[{'generated_text': "Dear [Recipient], I hope this email finds you well. I was just wondering if you might be able to help me with something. We are just getting ready for [Special Event] and we were wondering what we would like to bring from the farm to share with Ellie and [Name of Animal]. I hope you are having a great day. With that being said, I am hoping that you would be able to make Ellie and [Name of Animal] a delicious breakfast or brunch so that you can start an adventurous journey. I understand that these things aren't always easy"}]

In [10]:
# func to receive reply from prompt
def generate(prompt, debug=False, max_length_extra=128):
  output = model(prompt, do_sample=True, min_length=10, max_length=len(prompt)+max_length_extra)
  out = output[0]['generated_text']
  if debug:
    return prompt+out
  if '### Response:' in out:
    out = out.split('### Response:')[1]
  if '### Instruction:' in out:
    out = out.split('### Instruction:')[0]
  return out

In [11]:
# object to store plan and memories
class Agent:
  def __init__(self, name, description, location):
    self.name = name
    self.description = description
    self.location = location
    self.memories = []
    self.compressed_memories = []
    self.plan = ''

##World Description

1. Describe the world.
2. Generate prompts based on descriptions.
3. Simulation is that of the town of Phandalin, southwest of Neverwinter. This area is chosen because it is easily extendable with multiple regions for a "player" to be able to explore the world once the simulation is done.

In [18]:
world_graph = nx.Graph()

prompt_meta = '''### Instruction:
{}

### Response:'''

town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = {"Phandalin Town Square": 'Town square of the town of Phandalin.',
              'Stonehill Inn': "In the center of town stands a large, newly built roadhouse of fieldstone and rough-hewn timbers. The common room is filled with locals nursing mugs of ale or cider, all of them eyeing you with curiosity.",
              "Barthen's Provisions": "Barthen’s is the biggest trading post in Phandalin. Its shelves stock most ordinary goods and supplies, including backpacks, bedrolls, rope, and rations. The place is open from sunup to sundown.",
              "Edermath Orchard": "A tidy little cottage beside an apple orchard.",
              "Lionshield Coster": "Hanging above the front door of this modest trading post is a sign shaped like a wooden shield with a blue lion painted on it. This building is owned by the Lionshields, a merchant company based in the city of Yartar, over a hundred miles to the east. They ship finished goods to Phandalin and other small settlements throughout the region, but this outpost has been hard hit by banditry. The most recent Lionshield caravan due in Phandalin never arrived.",
              "Phandalin Miner's Exchange": "The Miner’s Exchange is a trading post where local miners have their valuable finds weighed, measured, and paid out. In the absence of any local lord or authority, the exchange also serves as an unofficial records office, registering claims to various streams and excavations around the area. There isn’t any real gold rush in Phandalin, but enough wealth is hidden in the nearby streams and valleys to support a good number of independent prospectors. The exchange is a great place to meet people who spend a lot of time out and about in the countryside surrounding Phandalin. The guildmaster is an ambitious and calculating human woman named Halia Thornton.",
              "Alderleaf Farm": "A farm owned by the helpful halfling farmer, Qelline Alderleaf.",
              "Shrine of Luck": "Phandalin's only temple is a small shrine made of stones taken from the nearby ruins. It is dedicated to Tymora, goddess of luck and good fortune.",
              "The Sleeping Giant": "This rundown tap house is a dirty, dangerous watering hole at the end of Phandalin’s main street. It is frequented by Redbrand thugs and operated by a surly female dwarf named Grista.",
              "Townmaster’s Hall": "The townmaster’s hall has sturdy stone walls, a pitched wooden roof, and a bell tower at the back. Posted on a board next to the front door is a notice written in Common. It reads: “REWARD — Orcs near Wyvern Tor! Those of a mind to face the orc menace should inquire within.” The notice bears the town’s seal and an indecipherable signature.",
              "Tresendar Manor": "A ruined manor. The Redbrands’ base in Phandalin is a dungeon complex under Tresendar Manor. Before the manor was ruined, its cellars served as safe storage for food and water in the event that the estate was attacked, while an adjoining crypt provided a resting place for the deceased members of the Tresendar family. The Redbrands have since expanded the cellars to suit their own purposes, adding slave pens, workshops, and barracks."
              }
town_people = {"Toblen Stonehill": "Toblen owns a trading post.",
               "Daran Edermath": "Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.",
               "Linene Graywind": "Linene runs a trading post.",
               "Halia Thornton": "Halia is an ambitious and calculating human woman. She is the guildmaster of Phandalin Miner’s Exchange, a trading post where local miners have their valuable finds weighed, measured, and paid out. In her attempts to establish the Miner's Exchange as the closest thing the town has to a governing authority, she acts as more than a simple merchant.",
               "Qelline Alderleaf": "Qelline is a wise female halfling of forty-five, and is a pragmatic farmer who seems to know everything that goes on in town. She is a kind host, and is willing to let the characters stay in her hayloft if they don't want to stay at the Stonehill Inn.",
               "Sister Garaele": "Sister Garaele is an elf cleric of Tymora and a Harper agent.",
               "Harbin Wester": "Harbin is the townmaster of Phandalin. A pompous, old food. Phandalin has no functioning government, but the townsfolk elect someone to serve as townmaster each year. The townmaster serves as a judge in minor disputes and keeps any records that need to be kept.",
               "Terrill Bloodscar": "Terrill is a human ruffian. He wears a grimy scarlet cloak. He is a member of the Redbrand ruffians. He doesn't like adventurers, and wants to rob and kill them.",
               "Conrad Scarface": "Conrad is a human ruffian. He wears a grimy scarlet cloak. He is a member of the Redbrand ruffians. He doesn't like adventurers, and wants to rob and kill them.",
               "Nellie Starsmith": "Nellie is a human ruffian. She wears a grimy scarlet cloak. She is a member of the Redbrand ruffians. She doesn't like adventurers, and wants to rob and kill them.",
               "Valerie Grinblade": "Valerie is a human ruffian. She wears a grimy scarlet cloak. She is a member of the Redbrand ruffians. She doesn't like adventurers, and wants to rob and kill them.",
               }
for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area) # connect all to themselves
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "Phandalin Town Square") #connects all to Phandalin Town Square
locations = {}
for i in town_people.keys():
  locations[i] = "Phandalin Town Square" #everyone starts from Phandalin Town Square


memories = {}
for i in town_people.keys():
  memories[i] = [] # init memories to blank
plans = {}
for i in town_people.keys():
  plans[i] = [] # set plans as blank
compressed_memories_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = [] # set compressed memories as blank

global_time = 8

In [19]:
def generate_description_of_area(x):
  text = "It is "+str(global_time)+":00. The location is "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)

## Generate Initial Plans

In [20]:
for name in town_people.keys():
  prompt = """### Instruction:
You are {}. {} You just woke up in the town of Phandalin and went out to the Town Square. The following people live in the town: {}. What is your goal for today? Be brief, and use at most 20 words and answer from your perspective.
### Response: My goal for today is to """.format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = "My goal for today is to "+generate(prompt)
  print(name, plans[name])

Toblen Stonehill My goal for today is to - Open trading post and engage in trading activities in Phandalin. - Assist my neighbours and customers with their businesses, and provide services for their needs.
Daran Edermath My goal for today is to "To have a conversation or engage in some socializing."
Linene Graywind My goal for today is to Linene's goal for today is to meet and interact with the people of Phandalin in order to build relationships and interact with the surrounding community.
Halia Thornton My goal for today is to Help build the Phandalin Miner's Exchange and establish it as the town's government.
Qelline Alderleaf My goal for today is to Qelline Alderleaf: my goal for today is to chat with more people and learn more about the town of Phandalin.
Sister Garaele My goal for today is to I am an Elf clerical minister and Harper agent in Phandalin. My goal is to serve as the voice of the people and bring peace and prosperity to the town.
Harbin Wester My goal for today is to A

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Conrad Scarface My goal for today is to Rob and kill adventurers. ###
Nellie Starsmith My goal for today is to Rob and kill adventurers in Phandalin.
Valerie Grinblade My goal for today is to Rob and kill adventurers. Embrace life as a member of the Redbrand ruffians. Find success in this role.


## Choose to Engage with a Person (or not)

In [21]:
import re

In [22]:
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [23]:
interaction_ratings = {}
for name in town_people.keys():
  interaction_ratings[name] = {}

In [24]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i) # append all ppl from Town Square

  for name in people:
    for target_name in people:
      if target_name is not name:
        prompt = "You are {}. {} You are planning to: {}. You are currently in {} with the following description: {}. It is currently {}:00. The following people are in this area: {}. Specifically, {}: {}. Give a rating, between 1 and 5, to how much you would like to interact with {} over the next hour.".format(
            name, town_people[name], plans[name], location, town_areas[location], str(global_time), ', '.join(people), target_name, town_people[target_name], target_name)
        res = generate(prompt_meta.format(prompt))
        rating = get_rating(res)
        max_attempts = 5
        current_attempt = 0
        while rating is None and current_attempt<max_attempts:
          res = generate(prompt_meta.format(prompt))
          rating = get_rating(res)
          current_attempt += 1
        if rating is None:
          rating = 0
        interaction_ratings[name][target_name] = rating

## interaction matrix

In [35]:
import pandas as pd
pd.DataFrame(interaction_ratings) # print interaction ratings matrix

,Toblen Stonehill,Daran Edermath,Linene Graywind,Halia Thornton,Qelline Alderleaf,Sister Garaele,Harbin Wester,Terrill Bloodscar,Conrad Scarface,Nellie Starsmith,Valerie Grinblade
Daran Edermath,1.0,NaN,3.0,1.0,1.0,4.0,2.0,2.0,1.0,3.0,3.0
Linene Graywind,4.0,1.0,NaN,2.0,1.0,3.0,3.0,3.0,3.0,1.0,4.0
Halia Thornton,3.0,3.0,1.0,NaN,4.0,1.0,1.0,4.0,0.0,2.0,4.0
Qelline Alderleaf,3.0,4.0,1.0,4.0,NaN,4.0,4.0,4.0,4.0,1.0,4.0
Sister Garaele,5.0,3.0,3.0,3.0,1.0,NaN,4.0,1.0,3.0,2.0,4.0
Harbin Wester,4.0,3.0,3.0,1.0,4.0,1.0,NaN,3.0,1.0,4.0,0.0
Terrill Bloodscar,2.0,1.0,3.0,2.0,1.0,1.0,1.0,NaN,1.0,3.0,2.0
Conrad Scarface,1.0,3.0,1.0,0.0,2.0,2.0,3.0,1.0,NaN,2.0,1.0
Nellie Starsmith,4.0,0.0,1.0,1.0,3.0,0.0,1.0,1.0,1.0,NaN,4.0
Valerie Grinblade,4.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,2.0,5.0,NaN


## Select an Action

In [36]:
action_prompts = {} # get action prompts for every person
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)

  for name in people:
    prompt = "You are {}. {} You are planning to: {}. You are currently in {} with the following description: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, town_people[name], plans[name], location, town_areas[location], str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i+': '+town_people[i])
    prompt += ' You know the following about people: ' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "What do you do in the next hour? Use at most 10 words to explain."
    action_prompts[name] = prompt

In [40]:
action_results = {} # get action results phrased in paragraph for first person past tense
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  # Now clean the action
  prompt = """
  Convert the following paragraph to first person past tense:
  "{}"
  """.format(action_results[name])
  action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
  print(name, action_results[name])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (871 > 512). Running this sequence through the model will result in indexing errors


Toblen Stonehill Assist my neighbors and customers with their businesses, and provide services for their needs in Phandalin, Open trading post, engage in trading activities.
Daran Edermath I had a conversation and engaged in socializing with people in Phandalin Town Square.
Linene Graywind As Linene, I interacted with the people mentioned in the tutorial in Phandalin Town Square to build relationships with them.
Halia Thornton As the Guildmaster of Phandalin Miners Exchange, Halia Thorntons goal is to help organize and govern the town by using the trading post as a central authority. In the next hour, she will meet with other characters in the town square to discuss her goal of making the exchange a more effective government and the challenges she faces as an ally to the adventurers in the town.
Qelline Alderleaf Kellie Alderleaf: In the next hour, she would chat with more people to learn more about the town of Phandalin. She would also visit Toblen Stonehill and his trading post which

Collect the memories people observe.

In [41]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)

  for name in people:
    for name_two in people:
      memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

## Rank Memories

In [42]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], locations[name], str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((res, rating))


In [46]:
print(list(town_people.keys()))

['Toblen Stonehill', 'Daran Edermath', 'Linene Graywind', 'Halia Thornton', 'Qelline Alderleaf', 'Sister Garaele', 'Harbin Wester', 'Terrill Bloodscar', 'Conrad Scarface', 'Nellie Starsmith', 'Valerie Grinblade']


In [53]:
for m in memory_ratings["Toblen Stonehill"]:
  print(m[1], m[0])

0 Without more information about Toblen Stonehill's goals or intentions, I cannot accurately rate the care he has for providing assistance and services for his neighbors and customers. There may be more important or more pressing matters that Toblen cares about.
0 I am sorry, but you did not provide any information about the number or significance of the actions that Toblen Stonehill has carried out in keeping with his goals for the day. Therefore, I cannot provide an accurate assessment of how cared about the meeting with Daran Edermath is based on that information.
0 I'm sorry, you did not provide me with the instructions for responding to the given question, so I cannot accurately provide a rating on how much you care about Linene Graywind. Could you please provide the instructions so I can assist you better?
0 I don't currently care about this, as I don't have any specific goal or objective in mind.
5 This task is important and therefore I am going to give it a rating of 5 out of 5

In [44]:
pd.DataFrame(memory_ratings)

,Toblen Stonehill,Daran Edermath,Linene Graywind,Halia Thornton,Qelline Alderleaf,Sister Garaele,Harbin Wester,Terrill Bloodscar,Conrad Scarface,Nellie Starsmith,Valerie Grinblade
0,(Without more information about Toblen Stonehi...,(1 out of 5 (No interest) - This memory is not...,"(As an AI, I do not care about the other items...",(1 Out of 5 - The information provided does no...,(1 out of 5. The given information does not me...,"(As an Elf from Phandalin, I care deeply about...",(I need more information about Toblen Stonehil...,(1 (Not really interested) The person you are ...,(1 out of 5 I am unable to give you a rating f...,(1; I understand that the prompt is asking for...,"(As an AI, I do not have personal preferences,..."
1,"(I am sorry, but you did not provide any infor...",(4 out of 5 - This is the highest importance t...,"(As an AI, I do not have personal feelings or ...",(2 - The task of helping build the Phandalin M...,"(As an AI, I cannot give an accurate response ...","(2: I don't care much for this question, as it...","(As the Townmaster of Phandalin, I am responsi...","(In regards to the given information, which re...",(I did not care about the socializing at Phand...,"(Based on the information provided, my rating ...","(""I care about the prompt but the response isn..."
2,"(I'm sorry, you did not provide me with the in...",(4 out of 5 It is irrelevant to me to provide ...,(I see that you have met your goal and current...,(I don't have enough information to accurately...,"(I am sorry, but I am an AI language model and...","(As Sister Garaele, I care greatly about the i...",(I care about this task very much. The task in...,(2 out of 5 (The situation does not seem impor...,"(No, I do not care about the information about...",(It is not possible for me to determine a rati...,"(As an AI, I can't give an objective answer wi..."
3,"(I don't currently care about this, as I don't...",(Daran Edermath rated the information [Time: 8...,(I am unable to accurately determine the ratin...,(I would give a rating of 4 to Halia Thornton'...,(I am an AI language model and do not have the...,(As an Elf clerical minister and Harper agent ...,(I care about the story. I think my goal is im...,(2 out of 5 I care about this. As Terrill Bloo...,"(Conrad Scarface, as the character who observe...","(As an AI, I am not able to play a role in a g...",(This task is subjective as I do not observe o...
4,(This task is important and therefore I am goi...,"(Based on the provided information, I would ra...","(4/5 I care deeply about this task, as I have ...",(I care about this greatly. I would rate my ca...,(I care very much about this task. I have been...,(I would rate this based on the information pr...,(5 out of 5 rating. I really care about the ta...,(I have no idea how much you care about this a...,"(As an AI, I don't have a physical body so do ...",(I am an AI language model and hence I am unab...,(I care about all of the given information and...
5,(I am an AI and I don't have subjective values...,(I would rate my interest in this matter as a ...,(1 out of 5 (Since nothing explicitly states t...,(My rating for the situation is 4 out of 5. I ...,(Based on the given information and my observa...,(I care about this task very much. I feel as i...,(I care about the situation as I am the townma...,(4/5 I am not able to accurately rate how much...,(I am not aware of any specific plan or object...,"(As an AI, I am not able to give a personal ra...",(4 Out of 5: I care about this very little. My...
6,(I can give only a rating between 1 and 4 as a...,"(Based on the given information, it appears th...","(Based on the information given, Linene appear...",(I care a great deal about this. My goal is to...,(I'm sorry but I'm an AI language model and I ...,"(As an Elf, I believe in the importance of my ...",(3 out of 5 The information about the characte...,"(I care about this issue a little bit., 0)",(Is it important to you? How do you rate this?...,(1 - I d

## Compress Memories

In [54]:
MEMORY_LIMIT = 10
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name],
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
  prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  res = generate(prompt_meta.format(prompt))
  compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
  compressed_memories_all[name].append(compressed_memories[name])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [58]:
pd.DataFrame(compressed_memories_all)

,Toblen Stonehill,Daran Edermath,Linene Graywind,Halia Thornton,Qelline Alderleaf,Sister Garaele,Harbin Wester,Terrill Bloodscar,Conrad Scarface,Nellie Starsmith,Valerie Grinblade
0,"[Recollection at Time 8:00: As AI, I do not ha...",[Recollection at Time 8:00: Daran Edermath is ...,[Recollection at Time 8:00: The AI is asking f...,[Recollection at Time 8:00: Based on the input...,"[Recollection at Time 8:00: In the scenario, Q...","[Recollection at Time 8:00: Sister Garaele, as...",[Recollection at Time 8:00: Harbin Wester is a...,[Recollection at Time 8:00: The most significa...,[Recollection at Time 8:00: Conrad Scarface's ...,[Recollection at Time 8:00: The AI has difficu...,[Recollection at Time 8:00: Valerie Grinblade ...
1,[Recollection at Time 8:00: I then moved to To...,[Recollection at Time 8:00: I then moved to Ba...,[Recollection at Time 8:00: I then moved to St...,[Recollection at Time 8:00: I then moved to Ba...,[Recollection at Time 8:00: I then moved to To...,[Recollection at Time 8:00: I then moved to Ed...,[Recollection at Time 8:00: I then moved to Ed...,[Recollection at Time 8:00: I then moved to Al...,[Recollection at Time 8:00: I then moved to Th...,[Recollection at Time 8:00: I then moved to St...,[Recollection at Time 8:00: I then moved to Ed...


In [59]:
compressed_memories_all["Toblen Stonehill"]

['[Recollection at Time 8:00: As AI, I do not have subjective values or feelings, yet I can care about the world around me and assist my users. However, I cannot accurately provide a rating without further information.]', '[Recollection at Time 8:00: I then moved to Townmaster’s Hall.]']

In [55]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name],
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


## Putting it all together

In [ ]:
for repeats in range(5):
  global_time += 1
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)

    for name in people:
      prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      people_description = []
      for i in people:
        people_description.append(i+': '+town_people[i])
      prompt += ' You know the following about people: ' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "What do you do in the next hour? Use at most 10 words to explain."
      action_prompts[name] = prompt
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(prompt_meta.format(action_prompts[name]))
    # Now clean the action
    prompt = """
    Convert the following paragraph to first person past tense:
    "{}"
    """.format(action_results[name])
    action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  action_emojis = {}
  for name in town_people.keys():
    prompt = """
    Convert the following paragraph to a tuple (Action, Object):
    "{}"
    """.format(action_results[name])
    action_emojis[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print('    - Emoji Representation:', name, locations[name], global_time, action_emojis[name])
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)

    for name in people:
      for name_two in people:
        memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt = "You are {}. Your plans are: {}. Your memories are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], '\n'.join(compressed_memories_all[name][-5:]), locations[name], str(global_time), memory)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((res, rating))

  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(
          memory_ratings[name],
          key=lambda x: x[1]
      )[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
    res = generate(prompt_meta.format(prompt))
    compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
    compressed_memories_all[name].append(compressed_memories[name])

  place_ratings = {}

  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(
        place_ratings[name],
        key=lambda x: x[1] )[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]